# 예측 1

라이브러리

In [31]:
# 공통
%matplotlib inline
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [32]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('전체1.csv', encoding='cp949')
# data = data[:-365]

# data = data.drop(['price'], axis = 1)
# data = data.drop(['전체_temp'], axis = 1)
# data = data.drop(['전체_hum'], axis = 1)

# data = data.drop(['강원_temp'], axis = 1)
# data = data.drop(['경기_temp'], axis = 1)
# data = data.drop(['경남_temp'], axis = 1)
# data = data.drop(['경북_temp'], axis = 1)
# data = data.drop(['광주_temp'], axis = 1)
# data = data.drop(['대구_temp'], axis = 1)
# data = data.drop(['대전_temp'], axis = 1)
# data = data.drop(['부산_temp'], axis = 1)
# data = data.drop(['서울_temp'], axis = 1)
# data = data.drop(['세종_temp'], axis = 1)
# data = data.drop(['울산_temp'], axis = 1)
# data = data.drop(['충남_temp'], axis = 1)
# data = data.drop(['충북_temp'], axis = 1)
# data = data.drop(['전남_temp'], axis = 1)
# data = data.drop(['전북_temp'], axis = 1)
# data = data.drop(['제주_temp'], axis = 1)
# data = data.drop(['인천_temp'], axis = 1)

# data = data.drop(['강원_hum'], axis = 1)
# data = data.drop(['경기_hum'], axis = 1)
# data = data.drop(['경남_hum'], axis = 1)
# data = data.drop(['경북_hum'], axis = 1)
# data = data.drop(['광주_hum'], axis = 1)
# data = data.drop(['대구_hum'], axis = 1)
# data = data.drop(['대전_hum'], axis = 1)
# data = data.drop(['부산_hum'], axis = 1)
# # data = data.drop(['서울_hum'], axis = 1)
# data = data.drop(['세종_hum'], axis = 1)
# data = data.drop(['울산_hum'], axis = 1)
# data = data.drop(['충남_hum'], axis = 1)
# data = data.drop(['충북_hum'], axis = 1)
# data = data.drop(['전남_hum'], axis = 1)
# data = data.drop(['전북_hum'], axis = 1)
# data = data.drop(['제주_hum'], axis = 1)
# data = data.drop(['인천_hum'], axis = 1)

data = data.drop(['소비 금액'], axis = 1)
data = data.drop(['평균온도'], axis = 1)
data = data.drop(['평균습도'], axis = 1)
# data = data.drop(['이용 횟수'], axis = 1)
data = data[:8760]

data = data.set_index('일자')
data = data.fillna(method='ffill')
data.tail()

,이용 횟수
일자,
2021-12-31 19:00,63.0
2021-12-31 20:00,72.0
2021-12-31 21:00,33.0
2021-12-31 22:00,48.0
2021-12-31 23:00,24.0


In [33]:
# 인덱스 지정(?)

# strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# strftime은 날짜와 시간(datetime)을 문자열로 출력
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
date_list = [start + relativedelta(hours = x) for x in range(0, len(data)) ]

data['index'] = date_list

# set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
data.set_index( ['index'], inplace=True) 

# index 출력 X
data.index.name=None
data.tail()

,이용 횟수
2021-12-31 19:00:00,63.0
2021-12-31 20:00:00,72.0
2021-12-31 21:00:00,33.0
2021-12-31 22:00:00,48.0
2021-12-31 23:00:00,24.0


In [34]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    data['이용 횟수'],
    order = (1,1,1),
    seasonal_order = (1,0,1,24),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3321      0.009     38.250      0.000       0.315       0.349
ma.L1         -1.0521      0.004   -260.503      0.000      -1.060      -1.044
ar.S.L24       1.0001      0.000   6109.753      0.000       1.000       1.000
ma.S.L24      -0.9764      0.002   -431.715      0.000      -0.981      -0.972
sigma2        83.7805      0.984     85.108      0.000      81.851      85.710


In [35]:
pred = results.get_prediction(
    start = pd.to_datetime('2021-01-01'),
    # 동적 회귀(?)
    dynamic = False
)

In [36]:
y_forecasted = pred.predicted_mean
y_truth = data['이용 횟수']['2021-01-01' : ]

# Compute the Mean Square Error: 평균 제곱 오차 계산
mse = (( y_forecasted - y_truth ) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format( round(mse, 2) ) ) 

The Mean Squared Error of our forecasts is 94.59


In [37]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 8760 )

In [38]:
forecast = pred_uc.predicted_mean
forecast

2022-01-01 00:00:00    17.792503
2022-01-01 01:00:00     9.075752
2022-01-01 02:00:00     4.048363
2022-01-01 03:00:00     1.832979
2022-01-01 04:00:00     1.268475
                         ...    
2022-12-31 19:00:00    60.151809
2022-12-31 20:00:00    50.006254
2022-12-31 21:00:00    41.856923
2022-12-31 22:00:00    34.249155
2022-12-31 23:00:00    23.746975
Freq: H, Name: predicted_mean, Length: 8760, dtype: float64

In [39]:
# 예측 데이터 csv로 저장
forecast.to_csv('소비금액만.csv', index=False)